In [35]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import r2_score

In [36]:
# load training data
Xtr_loadpath = 'Xtr.csv'
Xts_loadpath = 'Xts.csv'
ytr_loadpath = 'ytr.csv'

Xtr = np.loadtxt(Xtr_loadpath, delimiter=",")
Xts = np.loadtxt(Xts_loadpath, delimiter=",")
ytr = np.loadtxt(ytr_loadpath, delimiter=",")

In [37]:
# standardize the training data
Xtr_mean = np.mean(Xtr,axis=0)
Xtr_std = np.std(Xtr,axis=0)
Xtr_standardized = ((Xtr-Xtr_mean[None,:])/Xtr_std[None,:]) # revise this line as needed
Xts_standardized = ((Xts-Xtr_mean[None,:])/Xtr_std[None,:]) # revise this line as needed

# save the standardized training data
Xtr_savepath = 'Xtr_pytorch.csv'
Xts_savepath = 'Xts_pytorch.csv'
ytr_savepath = 'ytr_pytorch.csv'
yts_hat_savepath = 'yts_hat_pytorch.csv'

np.savetxt(Xtr_savepath, Xtr_standardized, delimiter=",")
np.savetxt(Xts_savepath, Xts_standardized, delimiter=",")
np.savetxt(ytr_savepath, ytr, delimiter=",")

In [38]:
# Convert the numpy arrays to PyTorch tensors
Xtr_torch = torch.Tensor(Xtr_standardized)
ytr_torch = torch.Tensor(ytr)
#Xts_torch = torch.Tensor(Xts)
#yts_torch = torch.Tensor(yts)

batch_size = 100  # size of each batch

# Create a training Dataset
train_ds = torch.utils.data.TensorDataset(Xtr_torch, ytr_torch)
# Creates a training DataLoader from this Dataset
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True) 

# Create a testing Dataset
#test_ds = torch.utils.data.TensorDataset(Xts_torch, yts_torch)
# Creates a testing DataLoader from this Dataset
#test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size) 

In [43]:
# create a model
# d_in = Xtr.shape[1]
# d_out = 1

# class DumbNet(nn.Module):
#     def __init__(self):
#         super(DumbNet, self).__init__()
#         self.Dense = nn.Linear(d_in,d_out)
#     def forward(self,x):
#         out = self.Dense(x)
#         return out

# model_d = DumbNet()   

# Usually, we would train the model at this point. 
# But this is only a demo, so we'll use the randomly initialized weights.

# nin: dimension of input data
# nh: number of hidden units
# nout: number of outputs

# class Net(nn.Module):
#     def __init__(self,nin,nh,nout):
#         super(Net,self).__init__()
#         self.sigmoid = nn.Sigmoid()
#         self.Dense1 = nn.Linear(nin,nh)
#         self.Dense2 = nn.Linear(nh,nout)
        
#     def forward(self,x):
#         x = self.sigmoid(self.Dense1(x))
#         out = self.Dense2(x)
#         return out

# model = Net(nin=nin, nh=nh, nout=nout)

# print(str(model))

# Define the model

nin = Xtr.shape[1]
nout = 1
#nh = 256

model = nn.Sequential(
    nn.Linear(nin, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, nout)
)

print(str(model))

Sequential(
  (0): Linear(in_features=26, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=16, bias=True)
  (5): ReLU()
  (6): Linear(in_features=16, out_features=8, bias=True)
  (7): ReLU()
  (8): Linear(in_features=8, out_features=1, bias=True)
)


In [44]:
import torch.optim as optim

opt = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [41]:
# # training parameters
# n_epochs = 100   # number of epochs to run
# batch_size = 10  # size of each batch
# batch_start = torch.arange(0, len(X_train), batch_size)

 
# training loop
# for epoch in range(n_epochs):
#     model.train()
#     with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
#         bar.set_description(f"Epoch {epoch}")
#         for start in bar:
#             # take a batch
#             X_batch = X_train[start:start+batch_size]
#             y_batch = y_train[start:start+batch_size]
#             # forward pass
#             y_pred = model(X_batch)
#             loss = loss_fn(y_pred, y_batch)
#             # backward pass
#             optimizer.zero_grad()
#             loss.backward()
#             # update weights
#             optimizer.step()
#             # print progress
#             bar.set_postfix(mse=float(loss))
#     # evaluate accuracy at end of each epoch
#     model.eval()
#     y_pred = model(X_test)
#     mse = loss_fn(y_pred, y_test)
#     mse = float(mse)
#     history.append(mse)
#     if mse < best_mse:
#         best_mse = mse
#         best_weights = copy.deepcopy(model.state_dict())

In [46]:
# training the model
num_epoch = 50
#batch_size = 10  # size of each batch

a_tr_loss = np.zeros([num_epoch])
#a_tr_accuracy = np.zeros([num_epoch])
#a_ts_loss = np.zeros([num_epoch])
#a_ts_accuracy = np.zeros([num_epoch])

a_tr_Rsq = np.zeros([num_epoch])

for epoch in range(num_epoch):    
    model.train() # put model in training mode
    
    # Initialize variables for tracking loss and R-squared score
    epoch_Rsq = 0.0
    batch_loss_tr = []
    # iterate over training set
    for train_iter, data in enumerate(train_loader):
        x_batch,y_batch = data
        y_batch = y_batch.view(-1,1)
        #y_batch = y_batch.type(torch.long)
        out = model(x_batch)
        # Compute Loss
        loss = criterion(out,y_batch.type(torch.float))
        batch_loss_tr.append(loss.item())
        # Compute gradients using back propagation
        opt.zero_grad()
        loss.backward()
        # Take an optimization 'step'
        opt.step()
        
        # Do hard classification: index of largest score
        #predicted = out.clamp(0,1).round().type(torch.long)
        # Compute number of decision errors
        #total += y_batch.size(0)
        #correct += (out == y_batch).sum().item()
        #epoch_Rsq += r2_score(y_batch.detach().numpy(), out.detach().numpy())
        
    epoch_Rsq /= len(train_loader)
    a_tr_loss[epoch] = np.mean(batch_loss_tr) # Compute average loss over epoch
    a_tr_Rsq[epoch] = epoch_Rsq
    
#     model.eval() # put model in evaluation mode
#     correct = 0 # initialize error counter
#     total = 0 # initialize total counter
#     batch_loss_ts = []
#     with torch.no_grad():
#         for data in test_loader:
#             images, labels = data
#             labels = labels.type(torch.long)
#             outputs = model(images)
#             batch_loss_ts.append(criterion(outputs,labels).item())
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
            
#     a_ts_loss[epoch] = np.mean(batch_loss_ts)
#     a_ts_accuracy[epoch] = 100*correct/total
    
    # Print details every print_mod epoch
    print('Epoch: {0:2d}   Train Loss: {1:.3f}   '.format(epoch+1, a_tr_loss[epoch])
          +'Train Acc: {0:.2f}   '.format(a_tr_Rsq[epoch]))
    
    # print('Epoch: {0:2d}   Train Loss: {1:.3f}   '.format(epoch+1, a_tr_loss[epoch])
    #       +'Train Acc: {0:.2f}    Test Loss: {1:.3f}   '.format(a_tr_accuracy[epoch], a_ts_loss[epoch])
    #       +'Test Acc: {0:.2f}'.format(a_ts_accuracy[epoch]))
    

Epoch:  1   Train Loss: 90.893   Train Acc: 0.00   
Epoch:  2   Train Loss: 80.343   Train Acc: 0.00   
Epoch:  3   Train Loss: 76.918   Train Acc: 0.00   
Epoch:  4   Train Loss: 75.239   Train Acc: 0.00   
Epoch:  5   Train Loss: 73.670   Train Acc: 0.00   
Epoch:  6   Train Loss: 72.648   Train Acc: 0.00   
Epoch:  7   Train Loss: 71.692   Train Acc: 0.00   
Epoch:  8   Train Loss: 71.278   Train Acc: 0.00   
Epoch:  9   Train Loss: 70.121   Train Acc: 0.00   
Epoch: 10   Train Loss: 69.344   Train Acc: 0.00   
Epoch: 11   Train Loss: 68.252   Train Acc: 0.00   
Epoch: 12   Train Loss: 67.849   Train Acc: 0.00   
Epoch: 13   Train Loss: 66.854   Train Acc: 0.00   
Epoch: 14   Train Loss: 66.031   Train Acc: 0.00   
Epoch: 15   Train Loss: 65.208   Train Acc: 0.00   
Epoch: 16   Train Loss: 64.655   Train Acc: 0.00   
Epoch: 17   Train Loss: 64.263   Train Acc: 0.00   
Epoch: 18   Train Loss: 63.639   Train Acc: 0.00   
Epoch: 19   Train Loss: 62.609   Train Acc: 0.00   
Epoch: 20   

KeyboardInterrupt: 